In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.neighbors import NearestNeighbors
from bramm_data_analysis import preprocessing
from bramm_data_analysis.matching import Matcher
import matplotlib.pyplot as plt

# Load Data

### Load Moss Data

In [ ]:
moss_data_path = Path("../data/Mines_2024.xlsx")
df_moss = preprocessing.MossPreprocessor(moss_data_path).load_preprocess()

### Load RMQS Data

In [ ]:
rmqs_data_path = Path("../data/RMQS.csv")
df_rmqs = preprocessing.RMQSPreprocessor(rmqs_data_path).load_preprocess()

In [ ]:
df_rmqs["date_complete"].dt.year.hist()

## Closest Points Matching

### Actual Matching

In [ ]:
data_matcher = Matcher()
matched_df = data_matcher.match_rmqs_to_moss(
    df_moss,
    df_rmqs,
)

In [ ]:
print(
    f"Threshold : {data_matcher.distance_threshold * 6371} km \n",
    f"Conserved : {matched_df.shape[0]} / {df_moss.shape[0]}",
)

### Matched Data Visualization

In [ ]:
plt.scatter(
    df_moss["longitude"], df_moss["latitude"], label="all moss", alpha=0.4
)
plt.scatter(
    matched_df[f"longitude{data_matcher.moss_suffix}"],
    matched_df[f"latitude{data_matcher.moss_suffix}"],
    label="Moss match",
    alpha=0.5,
    color="green",
)
plt.scatter(
    matched_df[f"longitude{data_matcher.rmqs_suffix}"],
    matched_df[f"latitude{data_matcher.rmqs_suffix}"],
    label="RMQS match",
    alpha=0.5,
    color="red",
)
plt.legend()
plt.plot()

## Multivariate Analysis

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstools as gs

verbose = True
graphics = True
gl.OptCst.define(gl.ECst.NTCOL, 6)
gdoc.setNoScroll()

In [ ]:
print(df_moss.columns)

In [ ]:
print(df_rmqs.filter(like="_tot_hf").columns)

In [ ]:
print(df_moss.columns)

In [ ]:
x1 = "longitude"
x2 = "latitude"
z1 = "aluminium"
z2 = "al_tot_hf"
raw_df = final_match.filter([x1, x2, z1, z2]).astype("float")
final_df = raw_df  # [
# (raw_df[z1] <= 15000)
# &
# (raw_df[z2] <= 1)
# ]

In [ ]:
mydb = gl.Db_fromPanda(final_df)
mydb.setLocators([x1, x2], gl.ELoc.X)
mydb.setLocator(z1, gl.ELoc.Z)

In [ ]:
mydb

In [ ]:
gl.dbStatisticsPrint(
    db=mydb,
    names=[z1, z2],
    opers=gl.EStatOption.fromKeys(["MEAN", "MINI", "MAXI"]),
    flagIso=False,
    title="Number of observations",
)

In [ ]:
ax = gp.correlation(mydb, namex=z1, namey=z2, asPoint=True, regrLine=True)
ax.decoration(title=f"Correlation between {z1} and {z2}")